# litfit — the shortest path from someone else's embedding to your task

litfit learns optimal linear projections from covariance statistics instead of gradient-based fine-tuning. One pass over your pairs, closed-form solution, done.

This notebook walks through the full pipeline on a text retrieval task (AskUbuntu duplicate detection). The same approach works for vision, multimodal, or any dense embeddings.

💻 [GitHub](https://github.com/b0nce/litfit) | 📦 [PyPI](https://pypi.org/project/litfit/)

In [19]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU: {gpu_name}")
else:
    print("⚠️  No GPU detected — litfit will still work, just slower.")
    print("   To enable GPU: Runtime → Change runtime type → T4 GPU")

!pip install -q litfit

✅ GPU: Tesla T4


## Step 1: Prepare your data

litfit needs three things:
- **embeddings** — vectors from any model (text, vision, multimodal)
- **ids** — a unique identifier per embedding (`torch.arange` is sometimes enough)
- **id_to_group** — a dict saying which ids are "similar" (duplicates, same class, relevant pairs)

We'll use the built-in AskUbuntu dataset: StackOverflow questions grouped by duplicate clusters.

In [ ]:
from litfit import encode_texts, load_askubuntu, split_data

SEED = 42

all_ids, all_texts, id_to_group = load_askubuntu(seed=SEED)
print(f"{len(all_ids)} questions, {len(set(id_to_group.values()))} duplicate groups")
print(f"\nExample: {all_texts[0][:120]}...")

# Encode with any HuggingFace model — swap this for your own embeddings
embs = encode_texts("intfloat/e5-base-v2", all_texts)
print(f"Embedding shape: {embs.shape}")

In [ ]:
# Group-aware split: all items in a group stay together (no leakage)
data = split_data(all_ids, all_texts, embs, id_to_group, seed=SEED)

train_ids, _, train_embs, _ = data["train"]
val_ids,   _, val_embs,   _ = data["val"]
test_ids,  _, test_embs,  _ = data["test"]

print(f"Train: {len(train_ids)} | Val: {len(val_ids)} | Test: {len(test_ids)}")

## Step 2: Measure the baseline

How well do raw embeddings perform before litfit touches them?

In [22]:
from litfit import evaluate_retrieval_fast

baseline = evaluate_retrieval_fast(test_embs, test_ids, id_to_group)
print(f"Baseline — R@1: {baseline['R@1']:.4f}  MAP@50: {baseline['MAP@50']:.4f}")

Baseline — R@1: 0.5360  MAP@50: 0.5084


## Step 3: Compute projections with litfit

The core pipeline:
1. Compute covariance matrices from positive pairs (sufficient statistics)
2. Generate ~40 candidate projections (fast mode) using different methods
3. Evaluate on the validation set with explore-exploit scheduling
4. Pick the best one

All closed-form — no iterative training.

In [ ]:
import time

from litfit import (
    compute_stats,
    evaluate_projections,
    find_dim_range,
    generate_fast_projections,
)

t0 = time.time()

# Sufficient statistics from training pairs
st = compute_stats(train_embs, train_ids, id_to_group)

# Automatically find useful dimensionality range
dim_fractions = find_dim_range(st, val_embs, val_ids, id_to_group)

# Generate and evaluate ~40 projection candidates
all_W = generate_fast_projections(st)
results, summary = evaluate_projections(
    all_W, val_embs, val_ids, id_to_group,
    test_embs=test_embs, test_ids=test_ids,
    dim_fractions=dim_fractions,
    seed=SEED,
)

print(f"Done in {time.time()-t0:.1f}s")

## Step 4: Extract the best projection and compare

In [24]:
import torch

from litfit import evaluate_retrieval_fast

# Find best config by validation MAP@50
flat = []
for key, dim_dict in results.items():
    for n_dims, scores in dim_dict.items():
        flat.append((key, n_dims, scores["MAP@50"]))
flat.sort(key=lambda x: x[2], reverse=True)
best_key, best_n_dims, best_val_score = flat[0]

# Apply projection to test embeddings
W = all_W[best_key]
projected = test_embs @ W
if best_n_dims is not None:
    projected = projected[:, :best_n_dims]

improved = evaluate_retrieval_fast(projected, test_ids, id_to_group)

dims_str = best_n_dims or test_embs.shape[1]
cfg_str = ", ".join(best_key[1:]) if len(best_key) > 1 else "default"

print(f"Best method: {best_key[0]} ({cfg_str})")
print(f"Dimensions:  {test_embs.shape[1]} → {dims_str}")
print()
print(f"{'':15s} {'R@1':>8s}  {'MAP@50':>8s}")
print(f"{'Baseline':15s} {baseline['R@1']:>8.4f}  {baseline['MAP@50']:>8.4f}")
print(f"{'+ litfit':15s} {improved['R@1']:>8.4f}  {improved['MAP@50']:>8.4f}")
print(f"{'Δ':15s} {improved['R@1']-baseline['R@1']:>+8.4f}  {improved['MAP@50']-baseline['MAP@50']:>+8.4f}")

Best method: Ray→AsymRef→MSE (reg=0.01, reg_mse=1.0, reg_refine=0.1)
Dimensions:  768 → 768

                     R@1    MAP@50
Baseline          0.5360    0.5084
+ litfit          0.5816    0.5771
Δ                +0.0456   +0.0687


## Step 5 (optional): Retrain on train+val for a final boost

Since litfit uses a closed-form solution, merging val into training data is safe — no risk of overfitting through early stopping leakage.

In [25]:
tv_embs = torch.cat([train_embs, val_embs], dim=0)
tv_ids = train_ids + val_ids

st_full = compute_stats(tv_embs, tv_ids, id_to_group)
all_W_full = generate_fast_projections(st_full, verbose=False)

W_full = all_W_full[best_key]
projected_full = test_embs @ W_full
if best_n_dims is not None:
    projected_full = projected_full[:, :best_n_dims]

final = evaluate_retrieval_fast(projected_full, test_ids, id_to_group)

print(f"{'':15s} {'R@1':>8s}  {'MAP@50':>8s}")
print(f"{'Baseline':15s} {baseline['R@1']:>8.4f}  {baseline['MAP@50']:>8.4f}")
print(f"{'Train only':15s} {improved['R@1']:>8.4f}  {improved['MAP@50']:>8.4f}")
print(f"{'Train+val':15s} {final['R@1']:>8.4f}  {final['MAP@50']:>8.4f}")

                     R@1    MAP@50
Baseline          0.5360    0.5084
Train only        0.5816    0.5771
Train+val         0.5822    0.5788


## Step 6: Export for inference

The projection is just a matrix multiply — export it as `torch.nn.Linear` and plug it into any pipeline.

In [26]:
import torch.nn as nn

out_dim = best_n_dims or W_full.shape[1]
layer = nn.Linear(W_full.shape[0], out_dim, bias=False)
layer.weight = nn.Parameter(W_full[:, :out_dim].T.cpu().float())

# Verify: should match train+val results
with torch.no_grad():
    check = layer(test_embs.cpu().float())
check_metrics = evaluate_retrieval_fast(check, test_ids, id_to_group)
print(f"Exported layer — R@1: {check_metrics['R@1']:.4f}  MAP@50: {check_metrics['MAP@50']:.4f}")
print(f"Layer shape: {layer.weight.shape} (in={W_full.shape[0]}, out={out_dim})")

Exported layer — R@1: 0.5822  MAP@50: 0.5788
Layer shape: torch.Size([768, 768]) (in=768, out=768)


## Bonus: Full sweep (800+ projections)

`generate_fast_projections` tests ~40 configs. If you have a GPU (or a bit of patience on CPU), `generate_all_projections` tries 800+ — including negative-pair statistics and more regularization variants. This can squeeze out extra performance.

In [ ]:
from litfit import compute_all_stats, generate_all_projections

t0 = time.time()

# Compute both positive and negative pair statistics
st_all, neg = compute_all_stats(train_embs, train_ids, id_to_group)

# Generate 800+ projection candidates
all_W_full_sweep = generate_all_projections(st_all, neg, include_neg_methods=True)
print(f"Generated {len(all_W_full_sweep)} projections in {time.time()-t0:.1f}s")

# Evaluate with the same dim range
results_full, summary_full = evaluate_projections(
    all_W_full_sweep, val_embs, val_ids, id_to_group,
    test_embs=test_embs, test_ids=test_ids,
    dim_fractions=dim_fractions,
    seed=SEED,
)

# Best result from full sweep
flat_full = []
for key, dim_dict in results_full.items():
    for n_dims, scores in dim_dict.items():
        flat_full.append((key, n_dims, scores["MAP@50"]))
flat_full.sort(key=lambda x: x[2], reverse=True)
best_key_full, best_n_dims_full, _ = flat_full[0]

W_sweep = all_W_full_sweep[best_key_full]
proj_sweep = test_embs @ W_sweep
if best_n_dims_full is not None:
    proj_sweep = proj_sweep[:, :best_n_dims_full]

sweep = evaluate_retrieval_fast(proj_sweep, test_ids, id_to_group)

dims_full_str = best_n_dims_full or test_embs.shape[1]

print(f"\nBest method: {best_key_full[0]} → {dims_full_str} dims")
print(f"\n{'':15s} {'R@1':>8s}  {'MAP@50':>8s}")
print(f"{'Baseline':15s} {baseline['R@1']:>8.4f}  {baseline['MAP@50']:>8.4f}")
print(f"{'Fast (~40)':15s} {improved['R@1']:>8.4f}  {improved['MAP@50']:>8.4f}")
print(f"{'Full (~800+)':15s} {sweep['R@1']:>8.4f}  {sweep['MAP@50']:>8.4f}")
print(f"\nTotal full sweep time: {time.time()-t0:.1f}s")

## Using litfit on your own data

Replace the dataset loading with your own embeddings and group labels:
```python
import numpy as np

# Your embeddings — any source, any modality
embs = np.load("my_embeddings.npy")  # shape (n, d)

# Your group labels — which items should be similar
ids = list(range(len(embs)))
id_to_group = {0: "A", 1: "A", 2: "B", 3: "B", 4: "C", ...}

# Then run the same pipeline from Step 3 onwards
```

litfit works with any dense vectors: text (e5, bge, MiniLM, OpenAI), vision (SigLIP, CLIP, DINOv2), multimodal, or your own custom model.